# OnlineStats.jl

https://github.com/joshday/OnlineStats.jl

Josh Day (`@joshday`)

In [60]:
using OnlineStats, Plots
gr()
map_rows = maprows

maprows (generic function with 1 method)

# Motivation 

![](http://www.ibmbigdatahub.com/sites/default/files/styles/xlarge-scaled/public/infographic_image/4-Vs-of-big-data.jpg?itok=4syrvSLX)

- Statisticians don't have tools to handle all of this
- Adapting our favorite algorithms is often nontrivial

# Types of problems
- Easy 
    - update the parameter directly (mean, variance)
- Hard
    - update "sufficient statistics" from which parameter is calculated (linear regression)
- Impossible
    - No analytical solution, must be approximated (quantiles, logistic regression)

# Introducing OnlineStats.jl

- Accepts input data piece by piece, not all at once
- Algorithms use O(1) memory
- Provides functionality for major areas of statistics
    - summary statistics to penalized regression

# Why Julia

- The same familiar story

# Toy Example

In [61]:
using OnlineStats
y = randn(100)

o = Mean()      # create

for yi in y
    fit!(o, yi) # update
end

# It works
mean(y) - mean(o)

-1.5265566588595902e-16

# Each OnlineStat is a type

In [62]:
# Construct objects with data
x = randn(100, 2)
Means(x)

■ Means{EqualWeight}
  >     value: [-0.06627022439964643,0.020480549943230213]
  >      nobs: 100


In [63]:
# Or create "empty" object, add data later
Means(2)

■ Means{EqualWeight}
  >     value: [0.0,0.0]
  >      nobs: 0


# Types are parameterized by Weight

- Last argument of constructor
- Default depends on whether problem is Easy/Hard vs. Impossible

In [64]:
Variance(EqualWeight())

■ Variance{EqualWeight}
  >     value: 0.0
  >      nobs: 0


- Consider online mean (many updates have this form):

$$\theta^{(t)} = (1 - \gamma_t) \; \theta^{(t-1)} + \gamma_t \; x_t$$

- For $\gamma_t = \frac{1}{t}$, updates are equivalent to offline mean
    - `EqualWeight`

- ExponentialWeight
    - $\gamma_t = \lambda$
- BoundedEqualWeight 
    - start as EqualWeight, shift to ExponentialWeight
    - $\gamma_t = \text{max}\left(\frac{1}{t}, \lambda\right)$
- LearningRate
    - Main use is for stochastic approximation
    - $\gamma_t = \text{max}\left(\frac{1}{t^r}, λ\right)$

In [1]:
o1 = EqualWeight()
o2 = ExponentialWeight(.2)
o3 = BoundedEqualWeight(.2)
o4 = LearningRate(.5)
ovec = [o1, o2, o3, o4]
map(OnlineStats.updatecounter!, ovec)

plt_wt = plot([0], map(OnlineStats.weight, ovec)', w = 3,
label = ["EqualWeight" "ExponentialWeight(.2)" "BoundedEqualWeight(.2)" "LearningRate(.5)"],
xlabel = "nobs", ylabel = "weight value", layout = 4, ylims = (0, 1))
for i in 1:50
    for o in ovec
        OnlineStats.updatecounter!(o)
    end
    push!(plt_wt, nobs(o1), map(OnlineStats.weight, ovec))
end

LoadError: LoadError: UndefVarError: EqualWeight not defined
while loading In[1], in expression starting on line 1

In [66]:
plt_wt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 
 
 
 nobs 
 
 
 weight value 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 62.5219,15.748 71.0628,86.4556 75.3332,110.025 79.6037,121.809 83.8741,128.88 88.1446,133.594 92.415,136.961 96.6855,139.486 100.956,141.45 105.226,143.022 
 109.497,144.307 113.767,145.379 118.038,146.285 122.308,147.062 126.579,147.735 130.849,148.325 135.12,148.845 139.39,149.307 143.66,149.72 147.931,150.092 
 152.201,150.429 156.472,150.735 160.742,151.015 165.013,151.271 169.283,151.507 173.554,151.724 177.824,151.926 182.095,152.113 186.365,152.287 190.635,152.449 
 194.906,152.601 199.176,152.744 203.447,152.878 207.717,153.004 211.988,153.123 216.258,153.235 220.529,153.341 224.799,153.442 229.07,153.537 233.34,153.628 
 237.61,153.714 241.881,153.796 246.151,153.874 250.422,153.949 254.692,154.021 258.963,154.089 263.233,154.154 267.504,154.217 271.774,154.277 276.045,154.335 
 280.315,154.39 
 "/>
 
 
 
 
 EqualWeight 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 
 
 
 nobs 
 
 
 weight value 
 
<polyline clip-path="url(#clip03)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 362.522,128.88 371.063,128.88 375.333,128.88 379.604,128.88 383.874,128.88 388.145,128.88 392.415,128.88 396.685,128.88 400.956,128.88 405.226,128.88 
 409.497,128.88 413.767,128.88 418.038,128.88 422.308,128.88 426.579,128.88 430.849,128.88 435.12,128.88 439.39,128.88 443.66,128.88 447.931,128.88 
 452.201,128.88 456.472,128.88 460.742,128.88 465.013,128.88 469.283,128.88 473.554,128.88 477.824,128.88 482.095,128.88 486.365,128.88 490.635,128.88 
 494.906,128.88 499.176,128.88 503.447,128.88 507.717,128.88 511.988,128.88 516.258,128.88 520.529,128.88 524.799,128.88 529.07,128.88 533.34,128.88 
 537.61,128.88 541.881,128.88 546.151,128.88 550.422,128.88 554.692,128.88 558.963,128.88 563.233,128.88 567.504,128.88 571.774,128.88 576.045,128.88 
 580.315,128.88 
 "/>
 
 
 
 
 ExponentialWeight(.2) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 
 
 
 nobs 
 
 
 weight value 
 
<polyline clip-path="url(#clip04)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 62.5219,215.748 71.0628,286.456 75.3332,310.025 79.6037,321.809 83.8741,328.88 88.1446,328.88 92.415,328.88 96.6855,328.88 100.956,328.88 105.226,328.88 
 109.497,328.88 113.767,328.88 118.038,328.88 122.308,328.88 126.579,328.88 130.849,328.88 135.12,328.88 139.39,328.88 143.66,328.88 147.931,328.88 
 152.201,328.88 156.472,328.88 160.742,328.88 165.013,328.88 169.283,328.88 173.554,328.88 177.824,328.88 182.095,328.88 186.365,328.88 190.635,328.88 
 194.906,328.88 199.176,328.88 203.447,328.88 207.717,328.88 211.988,328.88 216.258,328.88 220.529,328.88 224.799,328.88 229.07,328.88 233.34,328.88 
 237.61,328.88 241.881,328.88 246.151,328.88 250.422,328.88 254.692,328.88 258.963,328.88 263.233,328.88 267.504,328.88 271.774,328.88 276.045,328.88 
 280.315,328.88 
 "/>
 
 
 
 
 BoundedEqualWeight(.2) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 
 
 
 nobs 
 
 
 weight value 
 
<polyline clip-path="url(#clip05)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 362.522,215.748 371.063,257.168 375.333,275.517 379.604,286.456 383.874,293.92 388.145,299.431 392.415,303.713 396.685,307.165 400.956,310.025 405.226,312.444 
 409.497,314.525 413.767,316.34 418.038,317.942 422.308,319.368 426.579,320.65 430.849,321.809 435.12,322.865 439.39,323.831 443.66,324.72 447.931,325.542 
 452.201,326.304 456.472,327.013 460.742,327.676 465.013,328.297 469.283,328.88 473.554,329.429 477.824

In [67]:
y = -cumsum(rand(1000)) + 30 * randn(1000)
o1 = Mean(EqualWeight())
o2 = Mean(ExponentialWeight(.1))
plt_mean1 = plot([0], [0.0 0.0], label = ["EqualWeight" "ExponentialWeight"], w = 2, 
xlab = "time", ylab = "value", title = "changing model", legend = false)
map_rows(10, y) do yi
    fit!(o1, yi)
    fit!(o2, yi)
    push!(plt_mean1, nobs(o1), [mean(o1), mean(o2)])
end
scatter!(plt_mean1, y, markersize = 1);

y = 30 * randn(1000)
o1 = Mean(EqualWeight())
o2 = Mean(ExponentialWeight(.1))
plt_mean2 = plot([0], [0.0 0.0], label = ["EqualWeight" "ExponentialWeight"], w = 2, 
xlab = "time", ylab = "value", title = "constant model", legend = false)
map_rows(10, y) do yi
    fit!(o1, yi)
    fit!(o2, yi)
    push!(plt_mean2, nobs(o1), [mean(o1), mean(o2)])
end
scatter!(plt_mean2, y, markersize = 1);


# The Weight matters

In [68]:
plot(plt_mean1, plt_mean2, size = (800, 400))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 - 
 
 
 400 
 
 
 - 
 
 
 200 
 
 
 0 
 
 
 
 0 
 
 
 500 
 
 
 1000 
 
 
 
 
 
 changing model 
 
 
 time 
 
 
 value 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 71.516,82.8273 74.5141,88.1027 77.5121,83.2778 80.5102,82.8407 83.5082,84.9056 86.5063,87.2216 89.5043,90.403 92.5024,90.9818 95.5004,92.1023 98.4984,93.463 
 101.496,94.7496 104.495,95.3991 107.493,95.9591 110.491,97.3109 113.489,98.3146 116.487,99.8997 119.485,100.904 122.483,102.271 125.481,103.44 128.479,104.588 
 131.477,105.947 134.475,107.094 137.473,108.412 140.471,109.548 143.469,110.611 146.467,111.943 149.465,113.086 152.463,114.535 155.461,115.897 158.459,117.023 
 161.457,118.203 164.456,119.432 167.454,120.651 170.452,121.987 173.45,123.124 176.448,124.025 179.446,125.098 182.444,126.367 185.442,127.526 188.44,128.83 
 191.438,129.925 194.436,131.303 197.434,132.478 200.432,133.66 203.43,134.912 206.428,136.095 209.426,137.26 212.424,138.426 215.422,139.556 218.42,140.785 
 221.418,142.002 224.416,142.995 227.415,144.152 230.413,145.394 233.411,146.559 236.409,147.816 239.407,148.975 242.405,150.216 245.403,151.412 248.401,152.643 
 251.399,153.812 254.397,154.869 257.395,155.958 260.393,157.253 263.391,158.323 266.389,159.506 269.387,160.759 272.385,161.895 275.383,163.09 278.381,164.166 
 281.379,165.283 284.377,166.491 287.375,167.741 290.374,168.983 293.372,170.124 296.37,171.268 299.368,172.429 302.366,173.574 305.364,174.749 308.362,175.941 
 311.36,177.121 314.358,178.353 317.356,179.508 320.354,180.716 323.352,181.791 326.35,182.887 329.348,184.033 332.346,185.147 335.344,186.342 338.342,187.552 
 341.34,188.71 344.338,189.743 347.336,191 350.334,192.23 353.333,193.333 356.331,194.456 359.329,195.626 362.327,196.808 365.325,198.019 368.323,199.177 
 371.321,200.356 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 71.516,82.8273 74.5141,85.5644 77.5121,80.1658 80.5102,80.8511 83.5082,88.703 86.5063,92.9782 89.5043,102.01 92.5024,96.5125 95.5004,98.1696 98.4984,100.76 
 101.496,104.032 104.495,101.644 107.493,102.579 110.491,109.647 113.489,111.549 116.487,118.816 119.485,115.427 122.483,119.542 125.481,122.415 128.479,123.382 
 131.477,128.866 134.475,129.766 137.473,132.507 140.471,134.074 143.469,135.54 146.467,140.383 149.465,141.799 152.463,148.426 155.461,152.322 158.459,150.875 
 161.457,149.762 164.456,154.437 167.454,157.227 170.452,161.043 173.45,161.375 176.448,157.962 179.446,161.111 182.444,168.712 185.442,170.022 188.44,174.739 
 191.438,171.801 194.436,181.573 197.434,181.428 200.432,183.509 203.43,186.69 206.428,189.089 209.426,189.276 212.424,192.427 215.422,192.436 218.42,198.433 
 221.418,200.652 224.416,194.215 227.415,199.537 230.413,204.948 233.411,208.946 236.409,213.283 239.407,213.524 242.405,218.106 245.403,220.297 248.401,223.576 
 251.399,223.742 254.397,220.794 257.395,222.482 260.393,233.556 263.391,228.328 266.389,233.226 269.387,240.054 272.385,236.881 275.383,241.791 278.381,238.515 
 281.379,241.197 284.377,248.45 287.375,254.551 290.374,256.458 293.372,254.505 296.37,256.207 299.368,259.441 302.366,262.034 305.364,264.24 308.362,267.197 
 311.36,268.819 314.358,275.192 317.356,273.76 320.354,276.107 323.352,272.454 326.35,274.204 329.348,279.242 332.346,280.752 335.344,287.05 338.342,290.712 
 341.34,292.001 344.338,285.609 347.336,298.067 350.334,303.831 353.333,298.46 356.331,299.743 359.329,305.212 362.327,308.903 365.325,315.401 368.323,312.639 
 371.321,315.316 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

# Methods

- The following methods are available for every OnlineStat:
    - `fit!(o, data...)`
    - `nobs(o)`
    - `value(o)`
- Most types also have other methods:

In [69]:
o = LinReg(5, intercept = false)
coef(o)'

1x5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0

# Updating

In [70]:
y = randn(100)
o = Mean()

for yi in y
    fit!(o, yi)
end

fit!(o, y)      # simpler version of for loop

fit!(o, y, 10)  # update in batches of size 10

fit!(o, y, 0.1) # use update weight of .1 for each update

w = rand(100)
fit!(o, y, w)   # observation y[i] gets weight w[i]
;

# Why so many updating methods?

- Best seen through example
    - Linear regression using stochastic gradient descent
    - Plot of the loss on a test set over time

In [71]:
using DataGenerator
x, y, β = linregdata(1_000, 5);
x2, y2, β2 = linregdata(1_000, 5);

In [72]:
# "standard" update
o = StatLearn(size(x, 2), SGD(), intercept = false)
# plt_sgd1 = plot([0], coef(o)', lab = ["b$i" for i in 1:5]', xlab = "nobs", ylab = "value")
plt_sgd1 = plot([0], [loss(o, x2, y2)], xlab = "nobs", ylab = "loss", ylims = (0, 2))
map_rows(10, x, y) do xi, yi
    fit!(o, xi, yi); push!(plt_sgd1, nobs(o), [loss(o, x2, y2)])
end

# batch update
o = StatLearn(size(x, 2), SGD(), intercept = false)
# plt_sgd2 = plot([0], coef(o)', lab = ["b$i" for i in 1:5]', xlab = "nobs", ylab = "value")
plt_sgd2 = plot([0], [loss(o, x2, y2)], xlab = "nobs", ylab = "loss", ylims = (0, 2))
map_rows(10, x, y) do xi, yi
    fit!(o, xi, yi, 10); push!(plt_sgd2, nobs(o), [loss(o, x2, y2)])
end

# override weight
o = StatLearn(size(x, 2), SGD(), intercept = false)
# plt_sgd3 = plot([0], coef(o)', lab = ["b$i" for i in 1:5]', xlab = "nobs", ylab = "value")
plt_sgd3 = plot([0], [loss(o, x2, y2)], xlab = "nobs", ylab = "loss", ylims = (0, 2))
map_rows(10, x, y) do xi, yi
    fit!(o, xi, yi, .001); push!(plt_sgd3, nobs(o), [loss(o, x2, y2)])
end

In [73]:
# fit!(o, x, y)
plt_sgd1

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 
 
 
 nobs 
 
 
 loss 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 62.5219,64.4697 67.6998,-211.025 72.8777,32.1718 78.0557,-158.885 83.2336,238.06 88.4115,208.687 93.5895,236.862 98.7674,215.687 103.945,241.434 109.123,214.277 
 114.301,222.805 119.479,260.904 124.657,259.827 129.835,256.201 135.013,230.13 140.191,261.327 145.369,244.036 150.547,221.275 155.725,256.023 160.903,257.882 
 166.08,260.522 171.258,250.426 176.436,260.021 181.614,263.719 186.792,259.203 191.97,266.754 197.148,260.924 202.326,255.562 207.504,264.493 212.682,263.52 
 217.86,266.242 223.038,263.214 228.216,263.518 233.394,265.076 238.572,262.617 243.749,255.045 248.927,251.683 254.105,253.872 259.283,256.798 264.461,258.359 
 269.639,257.031 274.817,258.923 279.995,262.211 285.173,241.087 290.351,255.136 295.529,248.261 300.707,252.346 305.885,256.103 311.063,249.161 316.24,259.588 
 321.418,265.076 326.596,254.195 331.774,269.805 336.952,264.24 342.13,264.977 347.308,253.065 352.486,263.303 357.664,245.677 362.842,255.631 368.02,246.557 
 373.198,244.49 378.376,259.272 383.554,265.58 388.732,266.888 393.909,250.588 399.087,248.193 404.265,257.687 409.443,264.546 414.621,268.853 419.799,268.761 
 424.977,269.437 430.155,270.218 435.333,262.459 440.511,268.769 445.689,269.975 450.867,270.151 456.045,269.245 461.223,262.479 466.4,265.041 471.578,261.111 
 476.756,263.846 481.934,265.143 487.112,260.35 492.29,264.154 497.468,260.892 502.646,265.359 507.824,267.253 513.002,267.541 518.18,265.032 523.358,262.78 
 528.536,255.194 533.714,259.3 538.892,258.585 544.069,266.152 549.247,269.477 554.425,271.216 559.603,268.302 564.781,269.13 569.959,268.424 575.137,270.76 
 580.315,270.58 
 "/>
 
 
 
 
 y1

In [74]:
# fit!(o, x, y, 10)
plt_sgd2

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 
 
 
 nobs 
 
 
 loss 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 62.5219,64.4697 67.6998,18.3636 72.8777,191.307 78.0557,241.165 83.2336,236.962 88.4115,254.194 93.5895,262.179 98.7674,263.685 103.945,259.554 109.123,266.223 
 114.301,259.785 119.479,263.13 124.657,260.976 129.835,266.332 135.013,269.615 140.191,266.224 145.369,264.797 150.547,257.634 155.725,262.722 160.903,261.479 
 166.08,260.633 171.258,261.979 176.436,258.259 181.614,260.872 186.792,260.846 191.97,268.895 197.148,269.8 202.326,270.011 207.504,270.063 212.682,269.907 
 217.86,270.084 223.038,270.92 228.216,271.048 233.394,270.219 238.572,269.427 243.749,267.099 248.927,264.824 254.105,263.577 259.283,265.988 264.461,264.847 
 269.639,269.985 274.817,268.809 279.995,269.385 285.173,266.527 290.351,267.674 295.529,265.429 300.707,265.982 305.885,267.556 311.063,266.277 316.24,266.626 
 321.418,268.044 326.596,267.468 331.774,269.126 336.952,268.589 342.13,268.396 347.308,266.301 352.486,266.696 357.664,262.84 362.842,264.027 368.02,262.076 
 373.198,261.832 378.376,264.01 383.554,265.733 388.732,267.47 393.909,264.326 399.087,262.556 404.265,264.294 409.443,266.454 414.621,267.103 419.799,267.868 
 424.977,268.576 430.155,268.803 435.333,268.482 440.511,268.962 445.689,270.253 450.867,270.123 456.045,270.068 461.223,269.298 466.4,269.91 471.578,268.787 
 476.756,269.328 481.934,269.035 487.112,268.27 492.29,269.301 497.468,268.687 502.646,269.374 507.824,270.681 513.002,269.998 518.18,270.267 523.358,269.171 
 528.536,268.541 533.714,268.056 538.892,267.609 544.069,268.814 549.247,269.502 554.425,270.347 559.603,270.413 564.781,270.55 569.959,270.728 575.137,271.179 
 580.315,270.976 
 "/>
 
 
 
 
 y1

In [75]:
# fit!(o, x, y, .001)
plt_sgd3

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 
 
 
 nobs 
 
 
 loss 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 62.5219,64.4697 67.6998,67.6107 72.8777,72.3276 78.0557,76.2534 83.2336,80.3025 88.4115,83.1344 93.5895,86.8209 98.7674,89.2793 103.945,91.242 109.123,95.0669 
 114.301,98.967 119.479,105.795 124.657,114.003 129.835,116.032 135.013,120.446 140.191,123.381 145.369,124.899 150.547,127.784 155.725,129.519 160.903,130.95 
 166.08,133.613 171.258,137.772 176.436,142.762 181.614,143.97 186.792,146.398 191.97,148.938 197.148,149.657 202.326,152.371 207.504,154.789 212.682,156.841 
 217.86,159.188 223.038,160.507 228.216,162.282 233.394,164.713 238.572,169.911 243.749,171.201 248.927,175.119 254.105,177.019 259.283,178.775 264.461,181.377 
 269.639,181.756 274.817,183.404 279.995,183.603 285.173,185.407 290.351,186.114 295.529,186.407 300.707,189.475 305.885,192.039 311.063,192.641 316.24,193.678 
 321.418,195.105 326.596,196.267 331.774,196.671 336.952,197.536 342.13,197.543 347.308,198.322 352.486,198.89 357.664,199.561 362.842,201.09 368.02,201.422 
 373.198,202.27 378.376,203.471 383.554,204.734 388.732,207.055 393.909,208.405 399.087,209.117 404.265,210.544 409.443,211.799 414.621,212.667 419.799,213.506 
 424.977,214.474 430.155,215.222 435.333,216.216 440.511,216.791 445.689,217.878 450.867,218.707 456.045,219.399 461.223,221.126 466.4,223.326 471.578,223.88 
 476.756,224.872 481.934,225.61 487.112,225.722 492.29,226.08 497.468,227.247 502.646,227.588 507.824,227.969 513.002,229.191 518.18,230.772 523.358,230.828 
 528.536,232.781 533.714,233.594 538.892,234.319 544.069,234.962 549.247,235.503 554.425,236.121 559.603,236.608 564.781,237.117 569.959,238.064 575.137,238.597 
 580.315,239.151 
 "/>
 
 
 
 
 y1

# Algorithms

# Online MM

- First: offline MM (majorization - minimization)
    - objective $f(\theta)$ is hard to minimize
    - We create sequence of easier-to-minimize functions
    $$\begin{aligned}
        g_t(\theta^{(t)}) &= f(\theta^{(t)}) \\
        g_t(\theta) &\ge f(\theta)
    \end{aligned}$$
    

In [76]:
f(x) = x ^ 2
g(x, a) = f(x) + 0.5 * (x - a) ^ 2
gmin(a) = a / 3

θ = 3.
plt_mm = plot([f, x->g(x, θ)], -1, 4, w = 3, label = ["f" "g"])
vline!([θ], w = 3, label = "theta_old")
vline!([gmin(θ)], w = 3, label = "theta_new")
;

In [77]:
plt_mm

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 47.2441,350.825 52.6286,352.954 58.0132,354.972 63.3978,356.88 68.7823,358.677 74.1669,360.365 79.5514,361.942 84.936,363.409 90.3205,364.765 95.7051,366.011 
 101.09,367.147 106.474,368.173 111.859,369.088 117.243,369.893 122.628,370.588 128.012,371.173 133.397,371.647 138.782,372.011 144.166,372.264 149.551,372.408 
 154.935,372.441 160.32,372.364 165.704,372.176 171.089,371.878 176.473,371.47 181.858,370.952 187.243,370.323 192.627,369.585 198.012,368.735 203.396,367.776 
 208.781,366.706 214.165,365.526 219.55,364.236 224.934,362.835 230.319,361.324 235.703,359.703 241.088,357.972 246.473,356.13 251.857,354.178 257.242,352.116 
 262.626,349.943 268.011,347.66 273.395,345.267 278.78,342.763 284.164,340.15 289.549,337.426 294.934,334.591 300.318,331.647 305.703,328.592 311.087,325.427 
 316.472,322.151 321.856,318.766 327.241,315.27 332.625,311.663 338.01,307.947 343.395,304.12 348.779,300.183 354.164,296.135 359.548,291.978 364.933,287.71 
 370.317,283.331 375.702,278.843 381.086,274.244 386.471,269.535 391.856,264.715 397.24,259.786 402.625,254.746 408.009,249.596 413.394,244.335 418.778,238.964 
 424.163,233.483 429.547,227.892 434.932,222.19 440.317,216.378 445.701,210.456 451.086,204.423 456.47,198.28 461.855,192.027 467.239,185.664 472.624,179.19 
 478.008,172.606 483.393,165.912 488.778,159.107 494.162,152.193 499.547,145.168 504.931,138.032 510.316,130.786 515.7,123.431 521.085,115.964 526.469,108.388 
 531.854,100.701 537.239,92.9039 542.623,84.9965 548.008,76.9789 553.392,68.8509 558.777,60.6127 564.161,52.2642 569.546,43.8054 574.93,35.2363 580.315,26.557 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 47.2441,177.882 52.6286,184.35 58.0132,190.653 63.3978,196.79 68.7823,202.762 74.1669,208.569 79.5514,214.21 84.936,219.685 90.3205,224.996 95.7051,230.14 
 101.09,235.12 106.474,239.934 111.859,244.582 117.243,249.065 122.628,253.383 128.012,257.535 133.397,261.522 138.782,265.343 144.166,268.999 149.551,272.489 
 154.935,275.814 160.32,278.974 165.704,281.968 171.089,284.797 176.473,287.46 181.858,289.958 187.243,292.291 192.627,294.458 198.012,296.46 203.396,298.296 
 208.781,299.967 214.165,301.472 219.55,302.812 224.934,303.987 230.319,304.996 235.703,305.839 241.088,306.518 246.473,307.03 251.857,307.378 257.242,307.56 
 262.626,307.576 268.011,307.427 273.395,307.113 278.78,306.633 284.164,305.988 289.549,305.178 294.934,304.202 300.318,303.06 305.703,301.753 311.087,300.281 
 316.472,298.643 321.856,296.84 327.241,294.872 332.625,292.738 338.01,290.438 343.395,287.973 348.779,285.343 354.164,282.547 359.548,279.586 364.933,276.46 
 370.317,273.168 375.702,269.71 381.086,266.087 386.471,262.299 391.856,258.345 397.24,254.226 402.625,249.942 408.009,245.492 413.394,240.876 418.778,236.096 
 424.163,231.149 429.547,226.038 434.932,220.761 440.317,215.318 445.701,209.71 451.086,203.937 456.47,197.998 461.855,191.894 467.239,185.624 472.624,179.189 
 478.008,172.589 483.393,165.823 488.778,158.891 494.162,151.794 499.547,144.532 504.931,137.105 510.316,129.512 515.7,121.753 521.085,113.829 526.469,105.74 
 531.854,97.4851 537.239,89.0649 542.623,80.4793 548.008,71.7282 553.392,62.8118 558.777,53.7299 564.161,44.4825 569.546,35.0698 574.93,25.4916 580.315,15.748 
 
 "/>
 
 
 
 
 
 
 f 
 
 
 
 g 
 
 
 
 theta 
 
 
 old 
 
 
 
 theta 
 
 
 new

# Online/Stochastic MM

Use stochastic approximation of majorizing function

$$Q_t(\theta) = (1-\gamma_t)Q_{t-1}(\theta) + \gamma_t g_t(\theta)$$

$$\theta^{(t)} = \text{argmin}_\theta \; Q_t(\theta)$$

In [78]:
y = randn(100_000)
o = QuantileMM([.25, .5, .75])
fit!(o, y, 10)
value(o)

3-element Array{Float64,1}:
 -0.677774  
  0.00857336
  0.683189  

In [79]:
using Distributions
quantile(Normal(), [.25, .5, .75])

3-element Array{Float64,1}:
 -0.67449
  0.0    
  0.67449

# Stochastic Proximal Newton

- First: offline proximal newton algorithms
    $$f(\theta) + g(\theta)$$
    - $f(\theta) = \frac{1}{n} \sum_{i=1}^n f_i(\theta)$
    - $f_i$ convex/differentiable, $g$ convex (LASSO regression)

- Proximal Gradient Method minimizes (let $u = \theta^{(t-1)}$)
$$
f(u) + \nabla f(u)(\theta - u) + \frac{1}{2\gamma_t}(\theta - u)^T(\theta - u) + g(\theta)$$

- Proximal Newton Method uses a different quadratic term
$$
(\theta - u)^T H (\theta - u)
$$

# Stochastic Proximal Newton

Replace full gradient with noisy gradient

$$
f_i(u) 
+ \nabla f_i(u)(\theta - u) 
+ \frac{1}{2\gamma_t}(\theta - u)^T H_t(\theta - u)
+ g(\theta)$$


Use increasingly "worse" approximations as a mechanism for stabilizing iterations.

In [80]:
f(x) = x^2
df(x) = 2*x
plt_spn = plot(f, .5, 1.5, label = "noisy loss", w = 3)
c = 1.0
plot!(x -> f(c) + df(c)*(x - c) + 1/(2 *.1) * (x - c)^2, label = "gamma = .1", w = 3)
plot!(x -> f(c) + df(c)*(x - c) + 1/(2 *.01) * (x - c)^2, label = "gamma = .01", ylims = (0, 4), w = 3)
;

In [81]:
plt_spn

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 1.2 
 
 
 1.4 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 47.2441,350.148 52.6286,349.238 58.0132,348.31 63.3978,347.364 68.7823,346.399 74.1669,345.416 79.5514,344.416 84.936,343.397 90.3205,342.359 95.7051,341.304 
 101.09,340.23 106.474,339.139 111.859,338.029 117.243,336.9 122.628,335.754 128.012,334.589 133.397,333.407 138.782,332.206 144.166,330.986 149.551,329.749 
 154.935,328.493 160.32,327.22 165.704,325.928 171.089,324.618 176.473,323.289 181.858,321.943 187.243,320.578 192.627,319.195 198.012,317.794 203.396,316.374 
 208.781,314.937 214.165,313.481 219.55,312.007 224.934,310.515 230.319,309.005 235.703,307.476 241.088,305.93 246.473,304.365 251.857,302.781 257.242,301.18 
 262.626,299.561 268.011,297.923 273.395,296.267 278.78,294.593 284.164,292.901 289.549,291.19 294.934,289.461 300.318,287.715 305.703,285.949 311.087,284.166 
 316.472,282.365 321.856,280.545 327.241,278.707 332.625,276.851 338.01,274.977 343.395,273.084 348.779,271.174 354.164,269.245 359.548,267.298 364.933,265.333 
 370.317,263.349 375.702,261.347 381.086,259.328 386.471,257.29 391.856,255.233 397.24,253.159 402.625,251.066 408.009,248.955 413.394,246.826 418.778,244.679 
 424.163,242.514 429.547,240.33 434.932,238.128 440.317,235.908 445.701,233.67 451.086,231.414 456.47,229.139 461.855,226.846 467.239,224.535 472.624,222.206 
 478.008,219.859 483.393,217.493 488.778,215.109 494.162,212.708 499.547,210.287 504.931,207.849 510.316,205.392 515.7,202.918 521.085,200.425 526.469,197.914 
 531.854,195.384 537.239,192.837 542.623,190.271 548.008,187.687 553.392,185.085 558.777,182.464 564.161,179.826 569.546,177.169 574.93,174.494 580.315,171.801 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 47.2441,260.974 52.6286,263.631 58.0132,266.197 63.3978,268.672 68.7823,271.055 74.1669,273.348 79.5514,275.55 84.936,277.661 90.3205,279.681 95.7051,281.61 
 101.09,283.447 106.474,285.194 111.859,286.85 117.243,288.415 122.628,289.889 128.012,291.272 133.397,292.564 138.782,293.765 144.166,294.875 149.551,295.894 
 154.935,296.822 160.32,297.659 165.704,298.405 171.089,299.06 176.473,299.624 181.858,300.097 187.243,300.48 192.627,300.771 198.012,300.971 203.396,301.08 
 208.781,301.098 214.165,301.025 219.55,300.862 224.934,300.607 230.319,300.261 235.703,299.824 241.088,299.297 246.473,298.678 251.857,297.968 257.242,297.168 
 262.626,296.276 268.011,295.294 273.395,294.22 278.78,293.055 284.164,291.8 289.549,290.453 294.934,289.016 300.318,287.487 305.703,285.868 311.087,284.157 
 316.472,282.356 321.856,280.463 327.241,278.48 332.625,276.405 338.01,274.24 343.395,271.983 348.779,269.636 354.164,267.198 359.548,264.668 364.933,262.048 
 370.317,259.337 375.702,256.534 381.086,253.641 386.471,250.657 391.856,247.582 397.24,244.415 402.625,241.158 408.009,237.81 413.394,234.371 418.778,230.841 
 424.163,227.219 429.547,223.507 434.932,219.704 440.317,215.81 445.701,211.825 451.086,207.749 456.47,203.582 461.855,199.324 467.239,194.975 472.624,190.535 
 478.008,186.004 483.393,181.382 488.778,176.669 494.162,171.865 499.547,166.97 504.931,161.984 510.316,156.907 515.7,151.739 521.085,146.48 526.469,141.131 
 531.854,135.69 537.239,130.158 542.623,124.535 548.008,118.821 553.392,113.017 558.777,107.121 564.161,101.134 569.546,95.0563 574.93,88.8876 580.315,82.628 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#3cb370; stroke-width:2.4; stroke-opacity:1; fill:none" points="
 47.2441,-742.224 52.6286,-699.444 58.0132,-657.573 63.3978,-616.612 68.7823,-576.561 74.1669,-537.42 79.5514,-499.188 84.936,-461.867 90.3205,-425.455 95.7051,-389.953 
 101.09,-355.361 106.474,-321.679 111.859,-288.907 117.243,-257.044 122.628,-226.091 128.012,

Many SGD-like algorithms can be considered stochastic proximal newton:

- ADAGRAD, ADADELTA, ADAM, FOBOS, etc.

# StatLearn

Stochastic proximal newton algorithms for statistical learning

$$\frac{1}{n} \sum_{i=1}^n \ell_i(\theta) + g(\theta)$$

Objective is sum of 
- `Model` (linear regression) and
- `Penalty` (LASSO)

```julia
StatLearn{
    M <: Model, 
    P <: Penalty, 
    A <: Algorithm, 
    W <: Weight
}
```
- Trace plot examples used `L2Regression()` with `NoPenalty()` and `SGD()`

**StatLearn can fit any combination of `Model`, `Penalty`, and `Algorithm`**

- L2Regression, L1Regression, PoissonRegression, LogisticRegression, SVMLike, HuberRegression, QuantileRegression
- RidgePenalty, LassoPenalty, ElasticNetPenalty
- SGD, AdaGrad, RDA, AdaDelta, FOBOS, ADAM, MMGrad (experimental algorithm)

# Revisit stochastic proximal newton
$$
f_i(u) 
+ \nabla f_i(u)(\theta - u) 
+ \frac{1}{2\gamma_t}(\theta - u)^T H_t(\theta - u)
+ g(\theta)$$

In [82]:
o = StatLearn(5, LogisticRegression(), AdaGrad(), RidgePenalty(.1))

■ StatLearn
  > Intercept: 0.0
  >         β: [0.0,0.0,0.0,0.0,0.0]
  >     Model: OnlineStats.LogisticRegression()
  >   Penalty: OnlineStats.RidgePenalty(0.1)
  > Algorithm: OnlineStats.AdaGrad()
  >         η: 1.0
  >    Weight: OnlineStats.LearningRate
  >      Nobs: 0


In [83]:
x, y, β = linregdata(1_000_000, 500)  # 4GB
plot(β)  # coefficients are linspace(-1, 1, 500)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 - 
 
 
 1.0 
 
 
 - 
 
 
 0.5 
 
 
 0 
 
 
 0.5 
 
 
 1.0 
 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 47.2441,372.441 48.3124,371.726 49.3807,371.011 50.4489,370.296 51.5172,369.582 52.5855,368.867 53.6538,368.152 54.722,367.437 55.7903,366.722 56.8586,366.008 
 57.9269,365.293 58.9952,364.578 60.0634,363.863 61.1317,363.148 62.2,362.434 63.2683,361.719 64.3365,361.004 65.4048,360.289 66.4731,359.574 67.5414,358.859 
 68.6097,358.145 69.6779,357.43 70.7462,356.715 71.8145,356 72.8828,355.285 73.9511,354.571 75.0193,353.856 76.0876,353.141 77.1559,352.426 78.2242,351.711 
 79.2924,350.996 80.3607,350.282 81.429,349.567 82.4973,348.852 83.5656,348.137 84.6338,347.422 85.7021,346.708 86.7704,345.993 87.8387,345.278 88.9069,344.563 
 89.9752,343.848 91.0435,343.134 92.1118,342.419 93.1801,341.704 94.2483,340.989 95.3166,340.274 96.3849,339.559 97.4532,338.845 98.5215,338.13 99.5897,337.415 
 100.658,336.7 101.726,335.985 102.795,335.271 103.863,334.556 104.931,333.841 105.999,333.126 107.068,332.411 108.136,331.696 109.204,330.982 110.273,330.267 
 111.341,329.552 112.409,328.837 113.477,328.122 114.546,327.408 115.614,326.693 116.682,325.978 117.75,325.263 118.819,324.548 119.887,323.833 120.955,323.119 
 122.024,322.404 123.092,321.689 124.16,320.974 125.228,320.259 126.297,319.545 127.365,318.83 128.433,318.115 129.502,317.4 130.57,316.685 131.638,315.971 
 132.706,315.256 133.775,314.541 134.843,313.826 135.911,313.111 136.979,312.396 138.048,311.682 139.116,310.967 140.184,310.252 141.253,309.537 142.321,308.822 
 143.389,308.108 144.457,307.393 145.526,306.678 146.594,305.963 147.662,305.248 148.731,304.533 149.799,303.819 150.867,303.104 151.935,302.389 153.004,301.674 
 154.072,300.959 155.14,300.245 156.208,299.53 157.277,298.815 158.345,298.1 159.413,297.385 160.482,296.671 161.55,295.956 162.618,295.241 163.686,294.526 
 164.755,293.811 165.823,293.096 166.891,292.382 167.96,291.667 169.028,290.952 170.096,290.237 171.164,289.522 172.233,288.808 173.301,288.093 174.369,287.378 
 175.437,286.663 176.506,285.948 177.574,285.233 178.642,284.519 179.711,283.804 180.779,283.089 181.847,282.374 182.915,281.659 183.984,280.945 185.052,280.23 
 186.12,279.515 187.189,278.8 188.257,278.085 189.325,277.37 190.393,276.656 191.462,275.941 192.53,275.226 193.598,274.511 194.666,273.796 195.735,273.082 
 196.803,272.367 197.871,271.652 198.94,270.937 200.008,270.222 201.076,269.508 202.144,268.793 203.213,268.078 204.281,267.363 205.349,266.648 206.418,265.933 
 207.486,265.219 208.554,264.504 209.622,263.789 210.691,263.074 211.759,262.359 212.827,261.645 213.896,260.93 214.964,260.215 216.032,259.5 217.1,258.785 
 218.169,258.07 219.237,257.356 220.305,256.641 221.373,255.926 222.442,255.211 223.51,254.496 224.578,253.782 225.647,253.067 226.715,252.352 227.783,251.637 
 228.851,250.922 229.92,250.208 230.988,249.493 232.056,248.778 233.125,248.063 234.193,247.348 235.261,246.633 236.329,245.919 237.398,245.204 238.466,244.489 
 239.534,243.774 240.602,243.059 241.671,242.345 242.739,241.63 243.807,240.915 244.876,240.2 245.944,239.485 247.012,238.77 248.08,238.056 249.149,237.341 
 250.217,236.626 251.285,235.911 252.354,235.196 253.422,234.482 254.49,233.767 255.558,233.052 256.627,232.337 257.695,231.622 258.763,230.907 259.831,230.193 
 260.9,229.478 261.968,228.763 263.036,228.048 264.105,227.333 265.173,226.619 266.241,225.904 267.309,225.189 268.378,224.474 269.446,223.759 270.514,223.045 
 271.583,222.33 272.651,221.615 273.719,220.9 274.787,220.185 275.856,219.47 276.924,218.756 277.992,218.041 279.06,217.326 280.129,216.611 281.197,215.896 
 282.265,215.182 283.334,214.467 284.402,213.752 285.47,213.037 286.538,212.322 287.607,211.607 288.675,210.893 289.743,210.178 290.812,209.463 291.88,208.748

In [84]:
@time o = StatLearn(x, y, 1000, L2Regression(), LearningRate(.7), intercept = false);

LoadError: LoadError: UndefVarError: L2Regression not defined
while loading In[84], in expression starting on line 155

In [85]:
import MultivariateStats
@time β̂ = MultivariateStats.llsq(x, y, bias = false);

  3.595688 seconds (22 allocations: 1.916 MB)


In [86]:
# How good is our approximation?
@show maxabs(coef(o) - β̂)
plot(hcat(coef(o), β̂))

LoadError: LoadError: DimensionMismatch("dimensions must match")
while loading In[86], in expression starting on line 127

# Future Work

- OnlineStats assumes observations are in rows
    - ~~inefficient with column-major storage~~
    - Tim Holy has already fixed this?
- Not clear which types use approximations
- Add `Base.merge` methods to allow parallelization
- Easier plotting
- Stabilize experimental features
    - bootstrapping, hyperloglog